In [ ]:
# 杨树的注释版本转换

## 使用minimap2将v2 比对到v3, 然后读取v3的注释信息，将v2的注释信息转换到v3的坐标上

In [3]:
# 读取minimap2的paf输出
paf_file = "/home/xzg/v2tov3/v2_v3_mm2.paf"

v2_v3 = {}

for line in open(paf_file):
    line = line.strip().split()
    v2 = line[0]
    if v2 not in v2_v3:
        v2_v3[v2] = []
    v2_v3[v2].append(line)


In [5]:
v3_anno_txt = "/home/xzg/v2tov3/v3_tx_geneid.txt"
v3_anno = {}

for line in open(v3_anno_txt):
    line  = line.strip().split()
    v3_anno[line[0]] = line[1]


In [93]:
v2_to_v3 = {}
for k,v in v2_v3.items():
    genes = set()
    for item in v:
        tx = item[5]
        gene = v3_anno[tx]
        genes.add(gene)
    
    # 如果只比对到一个基因, 则直接赋值
    if len(genes) == 1:
        v2_to_v3[k] = list(genes)[0]
    
    # 如果比对到多个基因, 则选择最长的基因
    else:
        gene_len = {}
        for gene in genes:
            gene_len[gene] = 0
            for item in v:
                tx = item[5]
                if v3_anno[tx] == gene:
                    gene_len[gene] += int(item[10])
        v2_to_v3[k] = sorted(gene_len.items(), key=lambda x:x[1], reverse=True)[0][0]


In [102]:
v2_v3_pair = {}

In [103]:
for k,v in v2_to_v3.items():
    gene = k.split(".")[0]
    if gene not in v2_v3_pair:
        v2_v3_pair[gene] = set()
    v2_v3_pair[gene].add(v)

In [105]:
v2_v3_dict = {}

for k,v in v2_v3_pair.items():
    if len(v) == 1:
        v2_v3_dict[k] = list(v)[0]
    else:
        v2_v3_dict[k] = ",".join(list(v))

In [43]:
# save to file
with open("/home/xzg/v2tov3/v2_v3_pair.txt", "w") as f:
    for k,v in v2_v3_pair.items():
        v = set(v)
        if len(v) == 1:
            v = list(v)[0]
        else:
            v = ",".join(list(v))
        f.write(k+"\t"+ v +"\n")

In [78]:
# 读取xlsx 
import pandas as pd

xlsx_file = "/home/xzg/v2tov3/杨树茎细胞类型RNA-seq.xlsx"

df = pd.read_excel(xlsx_file)

In [107]:
df["v3_gene"] = df["gene_id"].map(v2_v3_dict)

In [108]:
df.head(10)

,gene_id,transcript_id(s),apex-1_FPKM,apex-2_FPKM,apex-3_FPKM,cambium-1_FPKM,cambium-2_FPKM,cambium-3_FPKM,metacambium-1_FPKM,metacambium-2_FPKM,...,GO Component,GO Function,GO Process,Blast nr,Nt,Swissprot,COG,Interpro,GO,v3_gene
0,POPTR_0014s19660,POPTR_0014s19660.1,15873.80,39876.17,45333.50,18842.85,21192.50,9386.02,18173.11,7888.83,...,-,-,-,gi|566205127|ref|XP_006375690.1|/0/hypothetica...,gi|87133043|gb|DQ371807.1|/0.0/Populus alba 18...,NaN,NaN,NaN,NaN,POPTR_005G133400v3
1,POPTR_0014s19780,POPTR_0014s19780.1,13400.07,46031.15,68894.36,22247.67,26835.85,13005.57,18082.63,7851.87,...,-,-,-,gi|566185232|ref|XP_006380095.1|/5.17753e-41/h...,gi|5912284|emb|AJ006440.1|/0.0/Populus trichoc...,NaN,NaN,NaN,NaN,POPTR_008G224500v3
2,POPTR_0019s07690,POPTR_0019s07690.1,13056.60,7076.44,4908.88,125.61,1.44,2.92,2729.84,3336.87,...,-,-,GO:0055086//nucleobase-containing small molecu...,gi|224144707|ref|XP_002325384.1|/1.40133e-166/...,gi|12658403|gb|AF330050.1|/0.0/Populus x gener...,sp|Q07469|BSPA_POPDE/9e-62/Bark storage protei...,CC2266/4e-21/---,PF01048/8.7E-25/Phosphorylase superfamily,biological_process:GO:0009116//nucleoside meta...,POPTR_019G050200v3
3,POPTR_0008s21830,POPTR_0008s21830.1,6413.87,29044.52,42378.48,6574.67,5835.63,2948.93,10239.97,5256.51,...,-,-,-,gi|566185230|ref|XP_006380094.1|/9.70703e-68/h...,gi|399611566|gb|JQ898624.1|/0.0/Populus deltoi...,NaN,NaN,NaN,NaN,POPTR_008G224500v3
4,POPTR_0009s00520,POPTR_0009s00520.1,5367.58,25195.31,36459.51,8178.47,9929.12,5030.42,8445.25,3718.27,...,-,-,-,gi|571443251|ref|XP_006576082.1|/5.84362e-111/...,gi|19919559|gb|AF479118.1|/0.0/Populus tremulo...,NaN,NaN,PTHR19446:SF230/7.3E-54/--,NaN,NaN
5,POPTR_0010s16030,POPTR_0010s16030.1,4493.04,2715.04,2219.00,3437.13,639.08,167.10,10680.57,12357.42,...,-,-,-,gi|224108812|ref|XP_002314976.1|/1.17854e-57/p...,gi|566191073|ref|XM_002314940.2|/0.0/Populus t...,sp|Q9LUV2|POP3_ARATH/2e-20/Probable protein Po...,NaN,PS51502/27.407/Stress-response A/B barrel doma...,NaN,POPTR_010G150500v3
6,POPTR_0009s11920,POPTR_0009s11920.1,2452.41,163.00,195.70,1207.53,999.91,615.10,1221.51,1383.02,...,-,GO:0005488//binding,-,gi|118489085|gb|ABK96349.1|/1.24189e-43/unknow...,gi|224077835|ref|XM_002305393.1|/3e-87/Populus...,sp|Q03250|RBG7_ARATH/5e-66/Glycine-rich RNA-bi...,XF0818/1e-17/---,PF00076/2.1E-23/RNA recognition motif. (a.k.a....,biological_process:GO:0006406//mRNA export fro...,POPTR_009G116400v3
7,POPTR_0013s07800,POPTR_0013s07800.1,2365.43,924.85,763.26,0.00,0.78,0.37,833.49,1309.55,...,-,-,GO:0055086//nucleobase-containing small molecu...,gi|566200037|ref|XP_006375994.1|/0/hypothetica...,gi|309838|gb|L20233.1|POPVEGSTRA/0.0/Populus x...,sp|Q07469|BSPA_POPDE/9e-117/Bark storage prote...,CC2266/3e-22/---,PF01048/1.0E-28/Phosphorylase superfamily,NaN,POPTR_013G080300v3
8,POPTR_0006s18240,POPTR_0006s18240.1,2251.02,2130.86,1399.60,4488.34,729.70,102.85,8823.37,9562.92,...,-,GO:0016787//hydrolase activity,GO:0044238,gi|566176865|ref|XP_006381792.1|/0/hypothetica...,gi|563440412|gb|KF541622.1|/0.0/Populus x cana...,sp|Q9FK75|GDL82_ARATH/6e-81/GDSL esterase/lipa...,NaN,PTHR22835/1.2E-119/--,NaN,POPTR_006G166800v3
9,POPTR_0004s16260,"POPTR_0004s16260.1,POPTR_0004s16260.2",2110.76,65.49,134.01,880.69,1061.76,841.27,1214.64,781.96,...,-,GO:0005488//binding;GO:0005488//binding,-,gi|224077836|ref|XP_002305429.1|/1.28658e-43/g...,gi|224077835|ref|XM_002305393.1|/0.0/Populus t...,sp|Q03878|GRP1_DAUCA/9e-64/Glycine-rich RNA-bi...,alr2311/2e-16/---,SM00360/2.6E-29/RNA recognition motif,biological_process:GO:0006406//mRNA export fro...,POPTR_004G155300v3


In [109]:
# 保存
df.to_excel("/home/xzg/v2tov3/杨树茎细胞类型RNA-seq.v3.xlsx", index=False)